In [1]:
%load_ext autoreload
%autoreload 2

from langchain_aws import BedrockLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
import os
import json
from langsmith import traceable

# Load the configuration from the config.json file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Set the state variables
os.environ["AWS_ACCESS_KEY_ID"] = config["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["AWS_SECRET_ACCESS_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_SECRET_KEY"]

import os

# Usage

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
openai_api_secret_key = os.getenv("OPENAI_API_KEY")



In [4]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

In [8]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain.chat_models import ChatOpenAI # Using OpenAI for simplicity, replace with Anthropic if needed.
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import requests
from bs4 import BeautifulSoup

# Wikipedia search tool
@tool
def wikipedia_search(query: str):
    """Searches Wikipedia for a given query and returns a summary."""
    try:
        url = f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"

        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        content = soup.find('div', class_='mw-parser-output')
        if content:
            paragraphs = content.find_all('p')
            summary = " ".join(paragraph.text for paragraph in paragraphs[:3]).strip()
            return summary
        else:
            return "Error: Content not found on Wikipedia."
    except requests.exceptions.RequestException as e:
        return f"Error: Request failed - {e}"
    except Exception as e:
        return f"Error: An unexpected error occurred - {e}"


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    # Check for tool calls in the *last response*, not the user's message.
    # if len(messages) > 1 and messages[-1].tool_calls:  # Check the AI's response
    #     return "tools"  # Go to tools node if the model suggests a tool call
    if len(messages) > 1 and "Error" not in messages[-1].content and len(messages[-1].content) > 100:
        return END  # Stop if a long enough, error-free summary is obtained
    else:
        return "tools" 

@traceable
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


In [9]:
from pydantic import BaseModel
from langchain_core.messages.ai import AIMessage


# The overall state of the graph (this is the public state shared across nodes)
class OverallState(BaseModel):
    cv : str
    summary : str
    main_skills : str
    pros : str
    cons : str




In [10]:

from src.ai.model import model, call_model
from src.ai.workflow import FlowState

def agentic_search(initial_state: FlowState):
    workflow = StateGraph(FlowState)

    workflow.add_node("agent", call_model)
    tool_node = ToolNode([wikipedia_search])
    workflow.add_node("tools", tool_node)  # Update as necessary to ensure it aligns with the rest of your infrastructure
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", should_continue)
    workflow.add_edge("tools", "agent")

    # checkpointer = MemorySaver()

    app = workflow.compile()
    # Run the agent with OpenAI
    final_state = app.invoke(
        initial_state)

    print(final_state["messages"][-1].content)


In [15]:
from src.ai.utils import load_outputs_placeholder, load_prompts, add_task_to_prompts, get_initial_state

storyline = "love"
flow_type = "search"
task = "Summarize the history of Python"


In [16]:
initial_state_outputs_placeholder = load_outputs_placeholder(storyline, flow_type)


initial_state = get_initial_state(storyline, flow_type, task)

KeyError: 'Summarize the history of Python'

In [ ]:
initial_state = FlowState(prompts=tasked_state_prompts, outputs=initial_state_outputs_placeholder)


In [11]:
agentic_search()

TypeError: __init__() takes 1 positional argument but 2 were given

In [4]:
workflow = StateGraph()

workflow.add_node('agent', call_model)
tool_node = ToolNode([wikipedia_search])
workflow.add_node('tool', tool_node)
workflow.add_node(END)

workflow.add_edge(START, 'agent')
workflow.add_conditional_edges('agent', should_continue)
workflow.add_edge("tools", "agent")

checkpointer = MemorySaver()

app = workflow.compile(checkpointer = checkpointer)

final_state = app.invoke({"messages": [HumanMessage(content="Summarize the history of Python")]})




In [ ]:
from langchain.chat_models import ChatOpenAI # Using OpenAI for simplicity, replace with Anthropic if needed.
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode